In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

# Konstanten
IMAGE_SIZE = (32, 32)
NUM_CLASSES = 2

# Bilddaten laden
image_directory = r"C:\Users\firas\Documents\Semester 2\Prototyping von KI-Systemen\Prototyping Projekt\Prototyping Projekt\preview2"
images = []
labels = []

for filename in os.listdir(image_directory):
    if filename.endswith('.jpeg'):
        img_path = os.path.join(image_directory, filename)
        img = load_img(img_path, target_size=IMAGE_SIZE)
        x = img_to_array(img)
        images.append(x)
        if 'True' in filename:
            labels.append(1)  # True label
        else:
            labels.append(0)  # False label

images = np.array(images) / 255.0
labels = np.array(labels)

# Modell erstellen
def create_model(optimizer, learning_rate):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    if optimizer == 'Adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'SGD':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(learning_rate=learning_rate)
    
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter-Bereich
batch_sizes = [1, 10 , 20]
epochs = [5, 10 , 15]  # Reduzierte Epochenanzahl für schnellere Grid-Search
optimizers = ['Adam', 'SGD']
learning_rates = [0.001, 0.01]  # Weniger Kombinationen für schnelleres Ergebnis

# Grid Search
best_accuracy = 0
best_params = {}

for batch_size in batch_sizes:
    for epoch in epochs:
        for optimizer in optimizers:
            for learning_rate in learning_rates:
                print(f'Training with batch_size={batch_size}, epochs={epoch}, optimizer={optimizer}, learning_rate={learning_rate}')
                
                model = create_model(optimizer, learning_rate)
                
                n_splits = 5  # Reduzierte Anzahl der Folds für schnellere Ergebnisse
                kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
                
                accuracy_per_fold = []
                
                for train_index, val_index in kf.split(images):
                    X_train, X_val = images[train_index], images[val_index]
                    y_train, y_val = labels[train_index], labels[val_index]

                    model.fit(X_train, y_train, batch_size=batch_size, epochs=epoch, validation_data=(X_val, y_val), verbose=0)
                    
                    scores = model.evaluate(X_val, y_val, verbose=0)
                    accuracy_per_fold.append(scores[1])
                
                avg_accuracy = np.mean(accuracy_per_fold)
                print(f'Average accuracy: {avg_accuracy}')
                
                if avg_accuracy > best_accuracy:
                    best_accuracy = avg_accuracy
                    best_params = {
                        'batch_size': batch_size,
                        'epochs': epoch,
                        'optimizer': optimizer,
                        'learning_rate': learning_rate
                    }

print('Beste Hyperparameter:')
print(best_params)
print(f'Beste Genauigkeit: {best_accuracy}')

# Beste Hyperparameter verwenden und endgültiges Modell trainieren
batch_size = best_params['batch_size']
epochs = 3  # Erhöhe die Anzahl der Epochen für das finale Training
optimizer = best_params['optimizer']
learning_rate = best_params['learning_rate']

model = create_model(optimizer, learning_rate)
model.fit(images, labels, batch_size=batch_size, epochs=epochs, verbose=1)

# Laden der Testdaten
test_data_directory = r"C:\Users\firas\Documents\Semester 2\Prototyping von KI-Systemen\Prototyping Projekt\Prototyping Projekt\validations_preview"
test_images = []
test_labels = []

for filename in os.listdir(test_data_directory):
    if filename.endswith('.jpeg'):
        img_path = os.path.join(test_data_directory, filename)
        img = load_img(img_path, target_size=IMAGE_SIZE)
        x = img_to_array(img)
        test_images.append(x)
        if 'True' in filename:
            test_labels.append(1)  # True label
        else:
            test_labels.append(0)  # False label

test_images = np.array(test_images) / 255.0
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Konfusionsmatrix berechnen und anzeigen
cm = confusion_matrix(test_labels, predictions)
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Konfusionsmatrix')
plt.ylabel('Tatsächliche Klasse')
plt.xlabel('Vorhergesagte Klasse')
plt.show()

# Metriken aus der Konfusionsmatrix
TP = cm[1, 1]
TN = cm[0, 0]
FP = cm[0, 1]
FN = cm[1, 0]

print(f'True Positives: {TP}')
print(f'True Negatives: {TN}')
print(f'False Positives: {FP}')
print(f'False Negatives: {FN}')



Training with batch_size=1, epochs=5, optimizer=Adam, learning_rate=0.001


C:\Users\firas\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Average accuracy: 0.990666663646698
Training with batch_size=1, epochs=5, optimizer=Adam, learning_rate=0.01
Average accuracy: 0.7586666703224182
Training with batch_size=1, epochs=5, optimizer=SGD, learning_rate=0.001
Average accuracy: 0.7586666703224182
Training with batch_size=1, epochs=5, optimizer=SGD, learning_rate=0.01
Average accuracy: 0.9613333344459534
Training with batch_size=1, epochs=10, optimizer=Adam, learning_rate=0.001
